In [57]:
import seaborn as sns
import pandas as pd
import pylab as plt
import numpy as np
import time
import os
import glob
import json
import tensorflow as tf
import sys
from IPython import display
from riotwatcher import RiotWatcher
import requests

api_file = os.path.join('..','..','apikeys','LOL_api_key.txt')
with open(api_file, 'r') as fin:
    watcher = RiotWatcher(fin.readline())

# Get matchids for masters** tier players
because there is no api call to just get a bunch matches I have to do some weird stuff. My strategy is: get challenger player list from the region -> getsummoner by playerid to get accountid -> get recent matches from each player -> check duplicates -> get match details

One note here is that the master tier isn't a constant size across regions unlike challenger. This is a possible source of bias in the data? Probably fine since there shouldn't be too too much difference across regions.

In [6]:
regions = ['RU', 'KR', 'BR1', 'OC1', 'JP1', 'NA1', 'EUN1', 'EUW1', 'TR1', 'LA1', 'LA2']
#regions = ['LA1', 'LA2']

#gameid_list = {}
#league_items = {}

#get last challenger games for each player in each region (purge duplicates)
#first loop through regions
for region in regions:
    
    #get all 'items' in the challenger league for solo queue (not sure about 'flex' queues?)
    #changed to master tier becuase that is the highest until mid january
    league_items[region] = watcher.league.masters_by_queue(region, 'RANKED_SOLO_5x5')['entries']
    
    #initialize gameid_list items
    gameid_list[region] = []

#loop through all items in each region's league to get matchid's
for region in regions:
    print(region)

    #get 100 matches from each player  
    for entry in league_items[region]:

        #convert player_id(aka summoner_id) to account_id
        account_id = watcher.summoner.by_id(region, entry['playerOrTeamId'])['accountId']

        #don't keep all matches in memory or on disk - we will get full match details later
        #queue = 420 is the code for 5x5_solo_queue
        match_list = watcher.match.matchlist_by_account(region,
                                     account_id,
                                     queue=420,
                                     begin_index = 1,
                                     end_index=100)['matches']
            
        #keep gameid_list in memory for later use and for easy purging of duplicates
        for match in match_list:
            gameid_list[region].append(match['gameId']) 

        #purge duplicates from gameid_list
        gameid_list[region] = list(set(gameid_list[region]))

       

LA1
LA2


In [8]:
with open(os.path.join('LOL_Masters_Data','2018-14-12.json'), 'w') as fp:
    json.dump(gameid_list, fp)

# Get match details

In [191]:
lol_df = pd.DataFrame()
region_list = []
gameid_load_list = []
with open(os.path.join('LOL_Data','masters_tier','2018-12-14.json'),'r') as fp:
    for key, item in json.load(fp).items():
        region_list.extend([key] * len(item))
        gameid_load_list.extend(item)
        
lol_df['region'] = region_list
lol_df['gameid'] = gameid_load_list
lol_df.describe()

,gameid
count,2.422160e+05
mean,2.469051e+09
std,1.140874e+09
min,1.898462e+08
25%,1.498358e+09
50%,2.886993e+09
75%,3.434856e+09
max,3.864771e+09


In [192]:
#reindex randomly so that we don't loop through regions sequentially
#this way we more efficiently use the api calls we're allotted
lol_df = lol_df.reindex(np.random.permutation(lol_df.index))

In [204]:
blue_win = []
blue_champs = []
red_champs = []
bad_element_list = []
region_list = []
gameid_list = []
for i,row in lol_df[137520:].iterrows():
    
    #NOTE: Summoner spell info is included in match_details but may be hard to keep linked to participants
    #NOTE 2: Timeline info. can give timeline info on many things for teams and individual participants
    #   this could be used for training  a real time win probability model.
    try:
        match_details = watcher.match.by_id(row['region'],row['gameid'])   
    except requests.exceptions.HTTPError as err:
        print(err)
        print('Error at row:',row['gameid'])
        print('element:',i)
        bad_element_list.append(i)
        continue
    
    #append region_list and gameid_list so we don't have to
    #deal with wrong bad region, gameid
    region_list.append(row['region'])
    gameid_list.append(row['gameid'])
    
    #blue win is true if teamid=100 (blue) and this same team won
    blue_win.append(int(match_details['teams'][0]['win']=='Win' and match_details['teams'][0]['teamId']==100))

    #build team champion lineups and append to columns
    blue_champ_row = []
    red_champ_row = []
    for participant in match_details['participants']:
        if participant['teamId'] == 100:
            blue_champ_row.append(participant['championId'])
        elif participant['teamId'] ==200:
            red_champ_row.append(participant['championId'])
        else:
            print('Illegal team value!',participant['teamId'])
            break
    blue_champs.append(blue_champ_row)
    red_champs.append(red_champ_row)    

404 Client Error: Not Found for url: https://la2.api.riotgames.com/lol/match/v3/matches/1531243518
Error at row: 1531243518
element: 230208
404 Client Error: Not Found for url: https://la1.api.riotgames.com/lol/match/v3/matches/2905085894
Error at row: 2905085894
element: 215061
503 Server Error: Service Unavailable for url: https://kr.api.riotgames.com/lol/match/v3/matches/3414197674
Error at row: 3414197674
element: 13598
404 Client Error: Not Found for url: https://la2.api.riotgames.com/lol/match/v3/matches/1505868301
Error at row: 1505868301
element: 239049
404 Client Error: Not Found for url: https://la1.api.riotgames.com/lol/match/v3/matches/2927745429
Error at row: 2927745429
element: 218132
404 Client Error: Not Found for url: https://la1.api.riotgames.com/lol/match/v3/matches/2904208066
Error at row: 2904208066
element: 216308
404 Client Error: Not Found for url: https://na1.api.riotgames.com/lol/match/v3/matches/616247377
Error at row: 616247377
element: 109191
404 Client Err

404 Client Error: Not Found for url: https://euw1.api.riotgames.com/lol/match/v3/matches/2909763352
Error at row: 2909763352
element: 186201
404 Client Error: Not Found for url: https://na1.api.riotgames.com/lol/match/v3/matches/628276717
Error at row: 628276717
element: 101251
404 Client Error: Not Found for url: https://eun1.api.riotgames.com/lol/match/v3/matches/3823690627
Error at row: 3823690627
element: 135413
404 Client Error: Not Found for url: https://la2.api.riotgames.com/lol/match/v3/matches/1531906682
Error at row: 1531906682
element: 231536
404 Client Error: Not Found for url: https://euw1.api.riotgames.com/lol/match/v3/matches/2919838489
Error at row: 2919838489
element: 179818
404 Client Error: Not Found for url: https://euw1.api.riotgames.com/lol/match/v3/matches/2910156766
Error at row: 2910156766
element: 186278
404 Client Error: Not Found for url: https://la2.api.riotgames.com/lol/match/v3/matches/1437357359
Error at row: 1437357359
element: 223737
503 Server Error: 

404 Client Error: Not Found for url: https://na1.api.riotgames.com/lol/match/v3/matches/628578321
Error at row: 628578321
element: 111918
404 Client Error: Not Found for url: https://la1.api.riotgames.com/lol/match/v3/matches/2905039018
Error at row: 2905039018
element: 218433
404 Client Error: Not Found for url: https://la2.api.riotgames.com/lol/match/v3/matches/1446809725
Error at row: 1446809725
element: 226292
429 Client Error: Too Many Requests for url: https://la2.api.riotgames.com/lol/match/v3/matches/610317897
Error at row: 610317897
element: 227884
404 Client Error: Not Found for url: https://tr1.api.riotgames.com/lol/match/v3/matches/3642190223
Error at row: 3642190223
element: 211424
404 Client Error: Not Found for url: https://la1.api.riotgames.com/lol/match/v3/matches/2927784087
Error at row: 2927784087
element: 219177
404 Client Error: Not Found for url: https://la1.api.riotgames.com/lol/match/v3/matches/2879472976
Error at row: 2879472976
element: 217181
404 Client Error

404 Client Error: Not Found for url: https://euw1.api.riotgames.com/lol/match/v3/matches/2910197784
Error at row: 2910197784
element: 153382
404 Client Error: Not Found for url: https://na1.api.riotgames.com/lol/match/v3/matches/616341343
Error at row: 616341343
element: 99135
404 Client Error: Not Found for url: https://la1.api.riotgames.com/lol/match/v3/matches/2853513597
Error at row: 2853513597
element: 215894
404 Client Error: Not Found for url: https://tr1.api.riotgames.com/lol/match/v3/matches/3642845407
Error at row: 3642845407
element: 211347
404 Client Error: Not Found for url: https://tr1.api.riotgames.com/lol/match/v3/matches/3566840994
Error at row: 3566840994
element: 204467
404 Client Error: Not Found for url: https://eun1.api.riotgames.com/lol/match/v3/matches/3818500767
Error at row: 3818500767
element: 146810
404 Client Error: Not Found for url: https://na1.api.riotgames.com/lol/match/v3/matches/629385508
Error at row: 629385508
element: 117188
404 Client Error: Not F

In [205]:
print(len(gameid_list)+len(bad_element_list))

104696


# Process match data and save to .csv

## Drop elements's from lol_df for which we got a HTTPError 
(all of these should be stored in bad_element_list). 
Not needed since we can just remake lol_df with the saved gameid+region lists

In [187]:
#drop bad gameid's from list
for bad_element in bad_element_list:
    lol_df = lol_df.drop(bad_element)

In [188]:
print(len(lol_df[:int(len(lol_df)/50)]))
print(len(bad_element_list))
print(len(blue_win))

4844
16
4828


## Process champion lists into np arrays.
Then add blue_win, blue_champ_array, and red_champ_array to lol_df.

In [206]:
#transform list of champion lists to np.array of champions.
#Shape is match x team (number of matches x 5 for each team)
blue_champ_array = np.zeros(shape=(len(blue_champs),5))
red_champ_array = np.zeros(shape=(len(red_champs),5))

if len(red_champ_array)!=len(blue_champ_array):
    pass

for i in range(len(blue_champ_array)):
    blue_champ_array[i] = blue_champs[i]
    red_champ_array[i] = red_champs[i]

print(blue_champ_array.shape)
print(red_champ_array.shape)

(104502, 5)
(104502, 5)


In [207]:
lol_df_processed = pd.DataFrame()

#Add radiant_win column to Dataframe
lol_df_processed['blue_win'] = np.array(blue_win)

#Add gameid
#Note: gameid can be duplicates across regions
lol_df_processed['gameid'] = gameid_list

#add region
lol_df_processed['region'] = region_list

#Add dire and radiant heroes to DataFrame under their own columns
for i in range(len(np.array(blue_champs)[1])):
    print(i)
    lol_df_processed['blue_champs'+str(i)] = blue_champ_array[:,i]
    lol_df_processed['red_champs'+str(i)] = red_champ_array[:,i]

0
1
2
3
4


In [208]:
#save data to csv file so we have it on disk
lol_df_processed.to_csv(os.path.join('LOL_data',
                                'masters_tier',
                                'saved'+
                                time.strftime('%Y-%m-%d')+
                                '.csv'))